<a href="https://colab.research.google.com/github/anash18/4105-HW-Nash/blob/main/ECGR_4105_HW5/ECGR_4105_HW5_Problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 2

In [644]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

In [645]:
#Calling the dataset
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/4105 Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [646]:
#Applying a binary map
#mapping the variables to a binary output
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist] = housing[varlist].apply(mapping)

housing.head()
#housing.dtypes

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1


In [647]:
#Splitting the Dataset
x2_train, x2_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)
x2_train.head()

#Splitting Variables
y2_train = x2_train.pop("price")
y2_val = x2_val.pop("price")

#print(x2_train)
#print(y2_train)
#x2_train.head()
#print(y2_val)

In [648]:
#Standardizing the Dataset

standard = StandardScaler()

vars2 = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

x2_train_stan = x2_train
x2_train_stan[vars2] = standard.fit_transform(x2_train_stan[vars2])

x2_val_stan = x2_val
x2_val_stan[vars2] = standard.fit_transform(x2_val_stan[vars2])
#x2_val_stan.head()

In [649]:
#Stacking them there inputs
x2_train_stan_numpy = np.c_[np.ones((len(y2_train), 1)), x2_train_stan[vars2]]
x2_val_stan_numpy = np.c_[np.ones((len(y2_val), 1)), x2_val_stan[vars2]]

#I am become tensor destroyer of deep learning
x2_train_ten = torch.tensor(x2_train_stan_numpy)
x2_val_ten = torch.tensor(x2_val_stan_numpy)
y2_train_ten = torch.tensor(y2_train.values)
y2_val_ten = torch.tensor(y2_val.values)


y2_train_mean = y2_train_ten.float().mean()
y2_train_std = y2_train.std()
y2_train_stan_ten = (y2_train_ten-y2_train_mean) / y2_train_std

y2_val_mean = y2_val_ten.float().mean()
y2_val_std = y2_val.std()
y2_val_stan_ten = (y2_val_ten-y2_val_mean) / y2_val_std

In [650]:
#Getting lost and starting my modeling career

#defining the model
def model(t_u, w5, w4, w3, w2, w1, b):
  return w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2 * t_u ** 2 + w1 * t_u + b

#defining the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#Setting up the initial parameters
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [651]:
#Time for an anime training arc

#defining a training loop
def training_loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:  # <1>
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

    t_p_val = model(t_u_val, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params


#SGD Number 1 learning rate of 0.00001
learning_rate_SGD1 = 0.00001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD1 = optim.SGD([params], lr=learning_rate_SGD1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD1,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26298384022938.2695, Validation loss: 24952661766042.1406

Epoch 3, Training loss: 26194525016140.6836, Validation loss: 24803363055944.6172

Epoch 500, Training loss: 24062939433895.2656, Validation loss: 22573483960989.0234

Epoch 1000, Training loss: 22756123929035.6523, Validation loss: 21339278009590.9805

Epoch 1500, Training loss: 21843127180528.6367, Validation loss: 20509121276043.1250

Epoch 2000, Training loss: 21155111802891.4375, Validation loss: 19897552080648.2695

Epoch 2500, Training loss: 20597387184874.6562, Validation loss: 19403807486485.5156

Epoch 3000, Training loss: 20116806891972.0781, Validation loss: 18973722858142.0977

Epoch 3500, Training loss: 19683494287205.6680, Validation loss: 18578605914666.5156

Epoch 4000, Training loss: 19280615772127.4688, Validation loss: 18203574011512.0859

Epoch 4500, Training loss: 18898644426215.6875, Validation loss:

tensor([-48864.2930, 208584.7812, 112123.6562, 284107.3438,  66783.8438,
        406606.4062], requires_grad=True)

In [652]:
#SGD Number 2 learning rate of 0.000001
learning_rate_SGD2 = 0.000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD2 = optim.SGD([params], lr=learning_rate_SGD2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD2,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26450699496010.7383, Validation loss: 25163104975345.4609

Epoch 3, Training loss: 26432463766474.8477, Validation loss: 25138293732889.8945

Epoch 500, Training loss: 25816266826436.4805, Validation loss: 24300517191106.7969

Epoch 1000, Training loss: 25583579044477.8672, Validation loss: 24067502831330.4727

Epoch 1500, Training loss: 25361566066086.8125, Validation loss: 23846163661663.0781

Epoch 2000, Training loss: 25149632902995.9102, Validation loss: 23635815709467.9805

Epoch 2500, Training loss: 24947218105140.0781, Validation loss: 23435815969162.5312

Epoch 3000, Training loss: 24753791938908.1484, Validation loss: 23245558738316.5000

Epoch 3500, Training loss: 24568854178124.2539, Validation loss: 23064472847024.2930

Epoch 4000, Training loss: 24391933114473.7969, Validation loss: 22892020640213.8398

Epoch 4500, Training loss: 24222582984116.8008, Validation loss:

tensor([-10967.3545,  67866.7578,  34569.5352,  42357.3281,  11724.2100,
         46130.7461], requires_grad=True)

In [653]:
#SGD Number 3 learning rate of 0.0000001
learning_rate_SGD3 = 0.0000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD3 = optim.SGD([params], lr=learning_rate_SGD3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD3,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26467849902406.6133, Validation loss: 25186369182519.4141

Epoch 3, Training loss: 26465931360333.7070, Validation loss: 25183770188196.2812

Epoch 500, Training loss: 26076414966550.4961, Validation loss: 24622265935758.8203

Epoch 1000, Training loss: 26014267910840.6953, Validation loss: 24517324364009.3711

Epoch 1500, Training loss: 25985816176485.2969, Validation loss: 24476436315070.3594

Epoch 2000, Training loss: 25960867480424.9219, Validation loss: 24447560498288.8438

Epoch 2500, Training loss: 25936377060317.9102, Validation loss: 24421738771705.8477

Epoch 3000, Training loss: 25912037334828.8984, Validation loss: 24396888610797.2500

Epoch 3500, Training loss: 25887816703088.1016, Validation loss: 24372424803655.4570

Epoch 4000, Training loss: 25863711352587.2305, Validation loss: 24348169022842.1484

Epoch 4500, Training loss: 25839720295977.0547, Validation loss:

tensor([3911.0151, 8931.4238, 4221.8862, 4584.6484, 1296.6592, 4712.9253],
       requires_grad=True)

In [654]:
#SGD Number 4 learning rate of 0.00000001
learning_rate_SGD4 = 0.00000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD4 = optim.SGD([params], lr=learning_rate_SGD4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD4,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469584131629.7969, Validation loss: 25188717802090.1836

Epoch 3, Training loss: 26469391310271.9219, Validation loss: 25188456703760.6602

Epoch 500, Training loss: 26383448958236.7461, Validation loss: 25071191739099.0586

Epoch 1000, Training loss: 26314139547672.2344, Validation loss: 24975039884209.2500

Epoch 1500, Training loss: 26258516916344.6836, Validation loss: 24896491823320.7109

Epoch 2000, Training loss: 26213777797980.1211, Validation loss: 24832112695778.1211

Epoch 2500, Training loss: 26177693020949.1211, Validation loss: 24779152575548.1328

Epoch 3000, Training loss: 26148490094012.0195, Validation loss: 24735408431843.8164

Epoch 3500, Training loss: 26124759721542.8164, Validation loss: 24699114002817.8086

Epoch 4000, Training loss: 26105381076696.1523, Validation loss: 24668851395778.4883

Epoch 4500, Training loss: 26089463092832.0273, Validation loss:

tensor([3787.0728, 1646.0212,  597.5497,  502.7557,  142.3710,  475.2841],
       requires_grad=True)

In [655]:
#Adam Number 1 learning rate of 0.1
learning_rate_Adam1 = 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam1 = optim.Adam([params], lr=learning_rate_Adam1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam1,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469755792759.0898, Validation loss: 25188951589767.7617

Epoch 3, Training loss: 26469734546773.3594, Validation loss: 25188924171014.7188

Epoch 500, Training loss: 26459239405601.7070, Validation loss: 25175374707548.1797

Epoch 1000, Training loss: 26448807537773.5859, Validation loss: 25161896650492.5273

Epoch 1500, Training loss: 26438495079175.9727, Validation loss: 25148562251103.6523

Epoch 2000, Training loss: 26428295243956.3789, Validation loss: 25135362563295.9805

Epoch 2500, Training loss: 26418202258984.5938, Validation loss: 25122289963608.9414

Epoch 3000, Training loss: 26408211382535.6406, Validation loss: 25109338158921.8945

Epoch 3500, Training loss: 26398318938087.5859, Validation loss: 25096502291642.8672

Epoch 4000, Training loss: 26388522112041.9883, Validation loss: 25083778596853.9297

Epoch 4500, Training loss: 26378818488697.1172, Validation loss:

tensor([493.4654, 496.2721, 497.6873, 499.9023, 499.9203, 499.8976],
       requires_grad=True)

In [656]:
#Adam Number 2 learning rate of 00.01
learning_rate_Adam2 = 0.01
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam2 = optim.Adam([params], lr=learning_rate_Adam2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam2,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469774914470.9961, Validation loss: 25188976267024.0977

Epoch 3, Training loss: 26469772789824.5859, Validation loss: 25188973525093.5547

Epoch 500, Training loss: 26468717483495.9102, Validation loss: 25187611564161.8594

Epoch 1000, Training loss: 26467657080150.0430, Validation loss: 25186242921438.7539

Epoch 1500, Training loss: 26466597886775.9375, Validation loss: 25184875733593.0977

Epoch 2000, Training loss: 26465539839935.3828, Validation loss: 25183509916952.3750

Epoch 2500, Training loss: 26464482870531.8906, Validation loss: 25182145377690.6133

Epoch 3000, Training loss: 26463426935187.1445, Validation loss: 25180782055690.1562

Epoch 3500, Training loss: 26462372027662.0078, Validation loss: 25179419942843.3633

Epoch 4000, Training loss: 26461318173104.6211, Validation loss: 25178059078026.6914

Epoch 4500, Training loss: 26460265217308.8008, Validation loss:

tensor([50.9264, 50.9527, 50.9667, 50.9916, 50.9918, 49.9974],
       requires_grad=True)

In [657]:
#Adam Number 3 learning rate of 0.001
learning_rate_Adam3 = 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam3 = optim.Adam([params], lr=learning_rate_Adam3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam3,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469776826646.1758, Validation loss: 25188978734752.5273

Epoch 3, Training loss: 26469776614171.3711, Validation loss: 25188978460546.2891

Epoch 500, Training loss: 26469671026314.9180, Validation loss: 25188842195410.2383

Epoch 1000, Training loss: 26469564814189.8945, Validation loss: 25188705123342.7305

Epoch 1500, Training loss: 26469458611165.6055, Validation loss: 25188568061782.4102

Epoch 2000, Training loss: 26469352417160.8398, Validation loss: 25188431010642.2305

Epoch 2500, Training loss: 26469246232744.5430, Validation loss: 25188293970484.6719

Epoch 3000, Training loss: 26469140057349.4805, Validation loss: 25188156940748.9414

Epoch 3500, Training loss: 26469033890974.7227, Validation loss: 25188019921434.4727

Epoch 4000, Training loss: 26468927733619.8242, Validation loss: 25187882912540.1523

Epoch 4500, Training loss: 26468821585282.7266, Validation loss:

tensor([5.9997, 5.9997, 5.9997, 5.9997, 5.9997, 4.9998], requires_grad=True)

In [658]:
#Adam Number 4 learning rate of 0.0001
learning_rate_Adam4 = 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam4 = optim.Adam([params], lr=learning_rate_Adam4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam4,
    params = params,
    t_u_train = x2_train_ten,
    t_c_train = y2_train_ten,
    t_u_val = x2_val_ten,
    t_c_val = y2_val_ten,
)

Epoch 1, Training loss: 26469777039121.0156, Validation loss: 25188979008958.8125

Epoch 2, Training loss: 26469777017871.1055, Validation loss: 25188978981535.0273

Epoch 3, Training loss: 26469776996621.2070, Validation loss: 25188978954111.2422

Epoch 500, Training loss: 26469766435460.9844, Validation loss: 25188965324538.7695

Epoch 1000, Training loss: 26469755810635.7891, Validation loss: 25188951612793.7656

Epoch 1500, Training loss: 26469745185900.9180, Validation loss: 25188937901153.0820

Epoch 2000, Training loss: 26469734561256.2148, Validation loss: 25188924189616.5938

Epoch 2500, Training loss: 26469723936701.7539, Validation loss: 25188910478184.3555

Epoch 3000, Training loss: 26469713312308.7266, Validation loss: 25188896766926.7539

Epoch 3500, Training loss: 26469702688005.9492, Validation loss: 25188883055773.4102

Epoch 4000, Training loss: 26469692063793.4141, Validation loss: 25188869344724.3242

Epoch 4500, Training loss: 26469681439671.1211, Validation loss:

tensor([1.5001, 1.5001, 1.5001, 1.5001, 1.5001, 0.5000], requires_grad=True)